In [16]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import

In [17]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np
from timeit import default_timer as timer

In [18]:
start = timer() # timer

In [19]:
# Define matrix sizes
size = 128
n = size*size

In [20]:
# create random input matrices
a = np.random.randn(size,size).astype(np.float32)
b = np.random.randn(size,size).astype(np.float32)

In [21]:
# Allocate memory on the device
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(b.nbytes)

In [22]:
# Copy input matrices to device memory
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)

## Main cuda kernel function

In [23]:
# Define the cuda kernel function for matrix multiplication
## The blockIdx and blockDim variables represent the index and dimensions of the block of threads 
#currently executing on the GPU, while threadIdx represents the index of the current thread within the block. 
#Multiplying the block index by the block dimension and adding the thread index gives the global index of 
#the current thread

mod = SourceModule("""
    __global__ void matrix_mul(float *a, float *b, float *c, int size) {
        int row = blockIdx.y * blockDim.y + threadIdx.y;
        int col = blockIdx.x * blockDim.x + threadIdx.x;
        float sum = 0.0;
        if (row < size && col < size) {
            for (int i = 0; i < size; i++) {
                sum += a[row*size + i] * b[i*size + col];
            }
            c[row*size + col] = sum;
        }
    }
""")


# Get a handle to the kernel function
matrix_mul = mod.get_function("matrix_mul")

In [24]:
# set block size and 
# set grid sizes for the GPU threads, based on the block size and size of the matrices
block_size = (16,16,1)
grid_size = (int(np.ceil(size/block_size[0])), int(np.ceil(size/block_size[1])),1)

In [25]:
# Call the kernel function with input matrices and output matrix
matrix_mul(a_gpu, b_gpu, c_gpu, np.int32(size), block=block_size, grid = grid_size)

In [26]:
# Allocate output memory on the host
c = np.zeros_like(a)

In [27]:
# Copy output matrix from device memory to host memory
cuda.memcpy_dtoh(c, c_gpu)

In [28]:
# Print input matrices and output matrix
print("Input matrix A:\n", a)
print("Input matrix B:\n", b)
print("Output matrix C:\n", c)


Input matrix A:
 [[-0.18593542  0.22338021  0.9246134  ...  1.4793277  -0.81670374
   1.2977253 ]
 [-2.1923976  -1.0449134   0.53853834 ...  0.9291856  -0.8984373
  -0.21670759]
 [ 1.1322881   0.53256243 -0.21414477 ...  1.2548547  -0.5611202
  -1.6145923 ]
 ...
 [ 1.0715582   1.0596999   1.1684084  ... -0.9959401  -1.6303049
  -0.1987654 ]
 [-0.7453522  -0.63320214 -0.5112142  ...  0.62949115  0.7362164
  -0.31776693]
 [ 0.10025843  0.7125079  -1.1531451  ...  0.3829878   0.95050937
  -1.3341309 ]]
Input matrix B:
 [[-0.37089834 -0.18000051 -0.5973354  ...  0.661749    0.33668098
   1.3884872 ]
 [ 0.21834491  0.9311182  -0.97298056 ...  0.95110714  0.8425769
  -2.1147854 ]
 [ 1.3106351  -0.67547995  0.9089664  ... -0.21392515 -0.9622833
   0.14477128]
 ...
 [-0.2453667   1.0754032  -0.5395588  ...  1.814994    0.93486553
  -1.6704469 ]
 [ 1.3035117  -0.5653697   1.0519493  ... -0.25452402  0.02995426
   1.6732676 ]
 [ 1.7481467   0.4807404   1.133023   ... -0.3796426  -0.8501197
  -0.

In [29]:
end_time = timer() - start
print("Multiplication time:", end_time)


Multiplication time: 0.0887345509999875
